<a href="https://colab.research.google.com/github/Rogerio-mack/Machine-Learning-I/blob/main/ML7_professor_resolve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="Universidade Presbiteriana Mackenzie">
</head>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->


# **Na prática: vamos criar modelos diferentes e selecionar um melhor modelos dentre eles?**
---



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.lines import Line2D
import seaborn as sns

# Penguins Class

* Plus: Hot Encode!

In [ ]:
from seaborn import load_dataset

df = load_dataset('penguins')
display(df.head())

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


In [ ]:
df.isna().sum()

species               0
island                0
bill_length_mm        2
bill_depth_mm         2
flipper_length_mm     2
body_mass_g           2
sex                  11
dtype: int64

In [ ]:
df = df.dropna(subset=['sex'])
df.isna().sum()

species              0
island               0
bill_length_mm       0
bill_depth_mm        0
flipper_length_mm    0
body_mass_g          0
sex                  0
dtype: int64

In [ ]:
# Veja aqui para uma solução com `make_column_transformer()` : https://datagy.io/sklearn-one-hot-encode/

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()

for cat_atributte in ['island','sex']:
  hot_values = ohe.fit_transform(df[[cat_atributte]])
  df[ohe.categories_[0]] = hot_values.toarray()
  df = df.drop(columns = [cat_atributte])

display(df.head())


,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,Biscoe,Dream,Torgersen,Female,Male
0,Adelie,39.1,18.7,181.0,3750.0,0.0,0.0,1.0,0.0,1.0
1,Adelie,39.5,17.4,186.0,3800.0,0.0,0.0,1.0,1.0,0.0
2,Adelie,40.3,18.0,195.0,3250.0,0.0,0.0,1.0,1.0,0.0
4,Adelie,36.7,19.3,193.0,3450.0,0.0,0.0,1.0,1.0,0.0
5,Adelie,39.3,20.6,190.0,3650.0,0.0,0.0,1.0,0.0,1.0


# Selecionando Classificadores



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import train_test_split
from sklearn import neighbors
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

X = df.drop(columns='species')
y = df.species

# Normalização
scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X) 

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=123)

save_estimators = []

base_estimators = [ LogisticRegression(),
                    neighbors.KNeighborsClassifier(),
                    DecisionTreeClassifier(),
                    RandomForestClassifier(),
                    GradientBoostingClassifier()]
                    
                               
param_grids = [ {},
                {'n_neighbors': [3,4,5], 'metric': ['euclidean','manhattan']},
                {'max_depth': [2,3,4,5]},
                {'n_estimators':[3,4,5,6]}, 
                {'n_estimators':[3,4,5,6]}]             
                
for i in range(len(base_estimators)):
  clf = GridSearchCV(base_estimators[i], param_grids[i], cv=5, scoring='accuracy')
  clf.fit(X_train, y_train)
  # print(clf.cv_results_)
  print(clf.best_estimator_)
  save_estimators.append(clf.best_estimator_)
  print()
  print("Detailed classification report:")
  print()
  y_pred = clf.predict(X_test)
  print(classification_report(y_test, y_pred))
  print()


LogisticRegression()

Detailed classification report:

              precision    recall  f1-score   support

      Adelie       0.98      1.00      0.99        44
   Chinstrap       1.00      0.95      0.97        20
      Gentoo       1.00      1.00      1.00        36

    accuracy                           0.99       100
   macro avg       0.99      0.98      0.99       100
weighted avg       0.99      0.99      0.99       100


KNeighborsClassifier(metric='euclidean')

Detailed classification report:

              precision    recall  f1-score   support

      Adelie       1.00      0.98      0.99        44
   Chinstrap       0.95      1.00      0.98        20
      Gentoo       1.00      1.00      1.00        36

    accuracy                           0.99       100
   macro avg       0.98      0.99      0.99       100
weighted avg       0.99      0.99      0.99       100


DecisionTreeClassifier(max_depth=5)

Detailed classification report:

              precision    recall  f

In [ ]:
[x.score(X_test, y_test) for x in save_estimators]

[0.99, 0.99, 0.97, 0.99, 0.98]

In [ ]:
best = np.array( [x.score(X_test, y_test) for x in save_estimators] ).argmax()
save_estimators[best]

LogisticRegression()